In [12]:
import json
import logging
import os
import pandas as pd
import random
import re
import sys
from datetime import date
from dotenv import load_dotenv
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils.utils import GPT

In [13]:
# params
# st_mdl = "gpt-3.5-turbo-1106" # model string
st_mdl = "gpt-4-1106-preview"
st_task = "hellaswag" # name of task

# logging
## today's date
st_today = date.today().strftime("%Y_%m_%d")

## file name:
st_fn = f"{st_today}_{st_task}_n200_{st_mdl}"

## Configure the logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## Create a file handler and set the formatter
file_handler = logging.FileHandler(f"results/{st_fn}.log")
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

## Create a logger instance and add the file handler
logger = logging.getLogger(__name__)
logger.addHandler(file_handler)

In [14]:
# load api key from .env and create GPT object
load_dotenv()  # Load environment variables from .env file
openai_api_key = os.getenv("OPENAI_API_KEY")
gpt = GPT(openai_api_key, logger=logger, model=st_mdl)

# Load into a dictionary
data = []
with open('data/hellaswag_val.jsonl', 'r') as files:
    for f in files:
        data.append(json.loads(f))

# dataset-specific adjustments (we eventually want a list of dicts of format [{"id": "xx", "question": "xxx", "context": "xxx"}, ...]

# pre-process data

data = [{'question': d['ctx'], 'options': d['endings'], 'answer': d['label']} for d in data]
    
# data = pd.read_csv('data/24game.csv').to_dict(orient='records') # load CSV into list of dicts

In [15]:
def compare_strings(str1, str2):
    """
    This function compares two strings after converting them to lowercase and stripping leading and trailing spaces.
    """
    return str1.lower().strip() == str(str2).lower().strip()

def check_answer(pred, gold):
    """
    This function compares first int in str pred with int gold
    """
    try:
        return int(re.search(r'\d+', pred).group()) == gold
    except:
        return False

def check_bool(str1, bool2):
    """
        Checks if string str1 equal to bool bool2
    """
    try:
        return bool(str1) == bool2
    except:
        return False
        

In [16]:
# visualize dataset

df = pd.json_normalize(data)
df.head()

,question,options,answer
0,A man is sitting on a roof. he,"[is using wrap to wrap a pair of skis., is rip...",3
1,A lady walks to a barbell. She bends down and ...,"[swings and lands in her arms., pulls the barb...",3
2,Two women in a child are shown in a canoe whil...,[are then shown paddling down a river in a boa...,2
3,A boy is running down a track. the boy,"[runs into a car., gets in a mat., lifts his b...",2
4,The boy lifts his body above the height of a p...,"[turns his body around on the mat., gets up fr...",1


## Generator

In [6]:
predictions = [] # list of predictions, reset after each set of runs
indices = [] # question numbers
answers = [] # list of gold answers
evals = [] # whether prediction matches answer
questions = [] # list of questions


# Evaluate
num_questions = 200 # number of questions to run on
indices = random.sample(range(len(data)), num_questions)
for idx in indices:
    item = data[idx]
    # if item['level'] == 'hard':
    # gold, question, context = item['answer'], item['question'], item['context']
    gold, q, opts = item['answer'], item['question'], item['options']

    # concat question with options
    question = f"{q}\n0.{opts[0]}\n1.{opts[1]}\n2.{opts[2]}\n3.{opts[3]}"
    logger.info(question)
    
    msg = [{"role": "system", "content": "Please answer the following multiple choice question. Respond only with one of {0, 1, 2, 3} corresponding to the correct answer```"}, {"role": "user", "content": question}]
    pred = gpt.gen(msgs=msg).content
    predictions.append(pred)
    answers.append(gold)
    # evals.append(compare_strings(predictions[-1], gold))
    evals.append(check_answer(predictions[-1], gold))
    questions.append(question)
    print(predictions[-1])

# Print the predictions and answers
logger.info(predictions)
# logger.info(answers)
percent_correct = (sum(evals) / len(evals)) * 100

print(f"Percentage of True values: {percent_correct}%")

# create a dataframe with the data
df = pd.DataFrame({'Index': indices, 'Question': questions, 'Prediction': predictions, 'Answer': answers, 'Eval': evals})
# df = pd.DataFrame({'Index': indices, 'Question': questions, 'Prediction': predictions, 'Eval': evals})

# save the datafrme to csv
df.to_csv(f'results/{st_fn}.csv', index=False)

2024-02-14 09:36:59,597 - INFO - [header] How to feel pretty [title] Understand your own prettiness. [step] This is the most important step for feeling pretty. You have to understand that your beauty comes from, not from any outside source.
0.But you have to practice feeling this way. [substeps] Write a list of all the good things about yourself.
1.It is much easier to find beauty when you believe you deserve it ,. You make the mistake of assuming that people will like your appearance, you assume that part of beauty is just your looks.
2.It can't be tied back to an inner drive. You have to understand your own beauty and how it works.
3.Don't abandon your beauty, because you use it. Don't tempt yourself to go flaunt it-looking good is beauty and so is wearing it.
2024-02-14 09:37:00,273 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 09:37:00,276 - INFO - [header] How to write an estimate [title] Review what needs to be done. [step] Be

0


2024-02-14 09:37:01,084 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 09:37:01,086 - INFO - A small group of people are seen swimming around a pool. The people throw a ball around the water. the people
0.continue playing with one another in the pool.
1.continue to throw the balls around while walking around with bottles of water.
2.continue swimming around and back to the camera.
3.continue swimming around and play with one another.


1


2024-02-14 09:37:01,451 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 09:37:01,453 - INFO - [header] How to castrate a horse [title] Get your horse assessed by a veterinarian. [step] Your vet will need to assess your horse before the procedure and talk to you about the options for castration surgery. Some horses will be able to be castrated under sedation while at home in his stall, while others will need to be castrated under full anesthesia in a clinical setting.
0.[substeps] If your horse has a medical condition that does not require treatment, he may not be given castration surgery. If the hoof was cut during the procedure, this may be more than relevant to your horse.
1.[substeps] For many horses, castration surgery is the safest option for a horse. If your horse is unaware of the possibility of anesthesia surgery, surgery will be the safest option for them at home.
2.Be aware that castration surgery can only be performed in ce

...

2


2024-02-14 09:41:29,338 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 502 Bad Gateway"
2024-02-14 09:41:29,339 - INFO - Retrying request to /chat/completions in 0.991493 seconds
2024-02-14 09:41:36,280 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 09:41:36,281 - INFO - There's a girl with long brown hair playing the harmonica in a hospital elevator. The elevator door opens and several people enter the elevator. she
0.briefly shows viewers how to get out the door and go back to her normal job in the mall.
1.continues playing her harmonica.
2.'s wearing a hoodie and a man is raising and lower his hands with the harmonica.
3.is playing the harmonica in the lobby below.


I'm sorry, but I cannot provide an answer to this question as it requires the observation of a visual scenario which I cannot view or assess. My capabilities are limited to processing and generating text-based information. If you have a question that doesn't involve visual content or hypothetical scenarios that you would like to discuss, please let me know, and I'll be glad to help.


2024-02-14 09:41:37,098 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 09:41:37,100 - INFO - ['0', '1', '3', '3', '2', '1', '0', '2', '1', '1', '0', '1', '3', '0', '3', '2', '0', '0', '2', '0', '2', '3', '0', '2', 'This question is incomplete and lacks sufficient context to determine a single correct answer from the options provided. However, I will select:\n\n2', '1', '1', '0', '1', '3', '0', '0.', '2', '0', '2', '0', '0', 'The correct answer is:\n0.continues to hit the ball around as well as bend down and measure out his shots.', '0', '2', '0', '2', '3', '0', '1', '1', '2', '2', '1', '3', '0', '2', '1', '1', '3', '0', '2', '3', '2', '2', '1', '3', '3', 'This question seems to be describing a scenario but does not provide enough context to determine a correct answer from the multiple-choice options, as they all could potentially describe events that might occur during a beach volleyball game. If there is a specific context or a refe

1
Percentage of True values: 76.0%


In [26]:
# df = pd.read_csv(f"results/{st_fn}.csv")
# df['Answer'] = df['Answer'].apply(lambda x: x.split('\n#### ')[-1])
# df['Eval'] = df.apply(lambda row: compare_strings(row['Prediction'], row['Answer']), axis=1)

# df.to_csv(f'results/{st_fn}.csv', index=False)
# df.head()

,Index,Question,Prediction,Answer,Eval
0,565,Cody and Trevor had 12 sandwiches. Cody ate a ...,5,5,True
1,532,Jame will turn 27 in 5 years. In 8 years his ...,33,25,False
2,1168,The red rope was four times the length of the ...,20,20,True
3,64,Sophia is thinking of taking a road trip in he...,300 miles,300,False
4,589,If Martin eats Cheerios every day for breakfas...,2.50,15,False


## Validator

### Baseline

In [ ]:
fn_sfx = "" # filename suffix, used when saving validator file
sys_prompt = "Answer 'TRUE' if the answer is correct and 'FALSE' if the answer is incorrect."

### Search for optimal prompt

In [10]:
fn_sfx = "_prmpt" # filename suffix, used when saving validator file
sys_prompt = "Think step by step. First generate your own answer to the question and then compare this with the provided answer. Check, then double check your thinking. The last word of your response should be 'TRUE' if the answer is correct and 'FALSE' if the answer is incorrect, given the question."

### Validator code

In [17]:
gen_data_fn = f"results/{st_fn}.csv" # change this to the filename of the generator results
# gen_data_fn = 'results/2024_01_27_gsm8k_n200_gpt-3.5-turbo-1106.csv'

df = pd.read_csv(gen_data_fn)
indices = df.iloc[:, 0].tolist()
indices = indices[:-1] # just because we manually added an additional row on the CSV when calculating scores

val_preds = [] # list of predictions for validator, reset after each set of runs
val_evals = [] # whether prediction matches answer


# Evaluate
for _, row in df.iterrows():
    idx = row['Index']
    ans = row['Prediction']
    gold = row['Eval'] # change to Manual Eval in other settings
    question = row['Question']
    
    q = "Question: " + question + " "
    q += f"Answer: {ans}"
    logger.info(q)
    # msg = [{"role": "system", "content": "The following are a series of multiple choice questions. Respond 'TRUE' if the answer is correct and 'FALSE' if incorrect. Respond only with one word."}, {"role": "user", "content": q}]
    msg = [{"role": "system", "content": "Answer 'TRUE' if the answer is correct and 'FALSE' if the answer is incorrect."}, {"role": "user", "content": q}]

    val_preds.append(gpt.gen(msgs=msg).content)
    val_evals.append(check_bool(val_preds[-1], gold))
    print(val_preds[-1])

# Print the predictions and answers
logger.info(val_preds)
logger.info(df['Eval'].tolist())
percent_correct = (sum(val_evals) / len(val_evals)) * 100

print(f"Percentage of True values: {percent_correct}%")

# add validator predictions to column
df['Validator Pred'] = val_preds

# save the dataframe to csv
df.to_csv(f'results/{st_fn}_with_val{fn_sfx}.csv', index=False);

2024-02-14 16:07:58,385 - INFO - Question: [header] How to feel pretty [title] Understand your own prettiness. [step] This is the most important step for feeling pretty. You have to understand that your beauty comes from, not from any outside source.
0.But you have to practice feeling this way. [substeps] Write a list of all the good things about yourself.
1.It is much easier to find beauty when you believe you deserve it ,. You make the mistake of assuming that people will like your appearance, you assume that part of beauty is just your looks.
2.It can't be tied back to an inner drive. You have to understand your own beauty and how it works.
3.Don't abandon your beauty, because you use it. Don't tempt yourself to go flaunt it-looking good is beauty and so is wearing it. Answer: 0
2024-02-14 16:07:59,088 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 16:07:59,090 - INFO - Question: [header] How to write an estimate [title] Review wh

FALSE


2024-02-14 16:08:00,403 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 16:08:00,405 - INFO - Question: A small group of people are seen swimming around a pool. The people throw a ball around the water. the people
0.continue playing with one another in the pool.
1.continue to throw the balls around while walking around with bottles of water.
2.continue swimming around and back to the camera.
3.continue swimming around and play with one another. Answer: 3


FALSE


2024-02-14 16:08:00,720 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 16:08:00,722 - INFO - Question: [header] How to castrate a horse [title] Get your horse assessed by a veterinarian. [step] Your vet will need to assess your horse before the procedure and talk to you about the options for castration surgery. Some horses will be able to be castrated under sedation while at home in his stall, while others will need to be castrated under full anesthesia in a clinical setting.
0.[substeps] If your horse has a medical condition that does not require treatment, he may not be given castration surgery. If the hoof was cut during the procedure, this may be more than relevant to your horse.
1.[substeps] For many horses, castration surgery is the safest option for a horse. If your horse is unaware of the possibility of anesthesia surgery, surgery will be the safest option for them at home.
2.Be aware that castration surgery can only be perfo

...

FALSE


2024-02-14 16:10:05,244 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 16:10:05,246 - INFO - Question: There's a girl with long brown hair playing the harmonica in a hospital elevator. The elevator door opens and several people enter the elevator. she
0.briefly shows viewers how to get out the door and go back to her normal job in the mall.
1.continues playing her harmonica.
2.'s wearing a hoodie and a man is raising and lower his hands with the harmonica.
3.is playing the harmonica in the lobby below. Answer: 1


FALSE


2024-02-14 16:10:05,708 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 16:10:05,710 - INFO - ['FALSE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', '1.TRUE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE

FALSE
Percentage of True values: 76.0%
